In [ ]:
import pygame, sys

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


參數設定

In [ ]:
speed = 20
WHITE = (255, 255, 255)

display_width = 500
display_height = 300


**擊球板物件**<br />
繼承自pygame的精靈物件pygame.sprite.Sprite<br />
將上下移動擊球板的程式段改為物件函式

In [ ]:
class Paddle(pygame.sprite.Sprite):
  def __init__(self,color,initial_position):
    pygame.sprite.Sprite.__init__(self)
    self.image = pygame.Surface([6,40])
    self.image.fill(color)
    self.rect=self.image.get_rect()
    self.rect.topleft=initial_position
    self.move_step = 5
      
  def move_up(self):
    if self.rect.y - self.move_step >= 0:
      self.rect.y -= self.move_step
          
  def move_down(self):
    if self.rect.y + self.rect.height + self.move_step <= display_height:
      self.rect.y += self.move_step

**乒乓球物件**<br />
相關全域參數移到物件內<br />
球的移動, 碰撞偵測等程式全部移到物件內<br />
新增調整球的移動速度函式

In [ ]:
class Ball(pygame.sprite.Sprite):
  def __init__(self,color,initial_position):
    pygame.sprite.Sprite.__init__(self)
    self.radius = 10
    self.image = pygame.Surface([self.radius*2, self.radius*2])
    self.image.fill(WHITE)
    self.image.set_colorkey(WHITE)
    self.color = color
    self.speed = speed
    self.dx = 10
    self.dy = 10
    self.x_dir = 1
    self.y_dir = 1
    pygame.draw.circle(self.image,self.color,(self.radius, self.radius), self.radius)
    self.rect=self.image.get_rect()
    self.rect.topleft=initial_position
      
  def move(self):
    self.rect.x += self.dx * self.x_dir
    self.rect.y += self.dy * self.y_dir
    
    if self.rect.x < self.radius:
      # game_over()
      self.x_dir *= -1

    if self.hit_back() or hit_paddle():
      self.x_dir *= -1

    if self.hit_sides():
      self.y_dir *= -1

  def increase_speed(self, step=1):
    self.dx += step
    self.dy += step
      
  def decrease_speed(self, step=1):
    if self.dx > 0:
      self.dx -= step
    if self.dy > 0:
      self.dy -= step
      
  def hit_back(self):
    if self.rect.x + self.rect.width >= display_width:
      self.rect.x = display_width - self.rect.width
      return True
    return False

  def hit_sides(self):
    if self.rect.y <= 0:
      self.rect.y = 0
      return True
    
    if self.rect.y + self.rect.height >= display_height:
      self.rect.y = display_height - self.rect.height
      return True
    return False

初始化pygame環境

In [ ]:
pygame.init()
clock = pygame.time.Clock()
display = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption("Let's Pong!")

建立擊球板及球的物件<br />
(將物件加入Group, 之後用 group.draw 可以一次將group內所有物件畫出)

In [ ]:
paddle = Paddle([255,0,0],[10, 40])
ball = Ball([0,255,0],[100,100])

group = pygame.sprite.Group()
group.add(paddle)
group.add(ball)

hit_paddle : 檢查球是否撞到板子(直接使用pygame.sprite的碰撞偵測函式)

In [ ]:
def hit_paddle():
  if pygame.sprite.collide_rect(paddle,ball):
    ball.rect.x = paddle.rect.x + paddle.rect.width
    return True
  return False

def game_over():
  pygame.quit()

程式主流程, 大部份程式都改呼叫物件函數處理<br />
新增按左右鍵可調整球的移動速度.

In [ ]:

while True:
  clock.tick(speed)  
  
  pygame.key.set_repeat(50)
  events = pygame.event.get()
  for event in events:
    if event.type == pygame.KEYDOWN:
      if event.key == pygame.K_w or event.key == pygame.K_UP:
        paddle.move_up()
      if event.key == pygame.K_s or event.key == pygame.K_DOWN:
        paddle.move_down()
      if event.key == pygame.K_LEFT:
        ball.decrease_speed()
      if event.key == pygame.K_RIGHT:
        ball.increase_speed()
      if event.key == pygame.K_q:
        game_over()
            
    if event.type == pygame.QUIT:
      game_over()
        
  ball.move()

  display.fill((0,0,0))
  group.draw(display)
  pygame.display.update()
  
  
  # from datetime import datetime
  # dateTimeObj = datetime.now()
  # timeStr = dateTimeObj.strftime("%H%M%S%f")
  # pygame.image.save(display, timeStr +".jpeg")
  

error: display Surface quit